# **Programming Assessment \#4**

Names: Go, Wilfred | Sibug, Jordan | Sy, James Matthew

More information on the assessment is found in our Canvas course. Link: https://dlsu.instructure.com/courses/93383/assignments/739602

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [1]:
import pandas
# load the words from the Gutenberg Document
import nltk
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()

from collections import Counter

# print("Extracting all documents from NLTK's Project Gutenberg Collection...")
all_words = Counter()
for filename in nltk.corpus.gutenberg.fileids():
    words = [word.lower() for word in nltk.corpus.gutenberg.words(filename)]
    all_words.update(words)
#   print("%s; tokens: %d; vocab: %d" % (filename, len(words), len(set(words))))

# print("Overall Statistics")
# total_tokens = sum(all_words.values())
total_types = len(all_words)

# print("Total tokens: %d" % total_tokens)
# print("Total vocabulary / type: %d" % total_types)
# print("Type/token ratio: %.4f" % (total_types / total_tokens))
# print("Vocabulary richness: %.4f" % (total_types / (total_tokens ** (1/2))))

[nltk_data] Downloading package gutenberg to C:\Users\JAMES
[nltk_data]     SY\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
# print the word
# print(set(words))
# print(words[500])
# print(words[100])
# print(words[664])
# print(words[32])
# print(words[10000])
# print(words[5])

# get the count of the word
# print(all_words[words[500]])

# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below. And while we discussed the general approach in class, kindly describe how you decided to implement the spell correction model. Include any modifications your group made as well. This might be a good spot to place part of your write up.*

In [3]:
# taken from http://norvig.com/spell-correct.html
def edits1(word):
    deletes = [] 
    deleteEdits = [] 
    transposes = [] 
    transposeEdits = []
    replaces = [] 
    replaceEdits = []
    inserts = []
    insertEdits = []
    "All edits that are one edit away from `word`."
    letters    = "abcdefghijklmnopqrstuvwxyz-\'"
#     [('', 'hte'), ('h', 'te'), ('ht', 'e'), ('hte', '')]
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
# DELETION
# '' + 'te' = 'te' 
# if L[-1] == '': ('>' + R[0])|('>') == >h|>
# 'h' + 'e' = 'he'  
# (L[-1] + R[0])|(L[-1]) == ht|h
# 'ht' + '' = 'ht'
# (L[-1] + R[0])|(L[-1]) = te|t
    for L,R in splits:
        if R:
            deletes.append(L + R[1:])
            if not L: 
                deleteEdits.append('>' + str(R[0]) + '|>')
            else:
                 deleteEdits.append(str(L[-1] + R[0]) + '|' + str(L[-1]))

# TRANSPOSITION
# [('', 'hte'), ('h', 'te'), ('ht', 'e'), ('hte', '')]
# ht|th R[0]+R[1] + '|' + R[1] + R[0]
#     transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    for L, R in splits:
        if len(R) > 1:
            transposes.append(L + R[1] + R[0] + R[2:])
            transposeEdits.append(R[0]+R[1] + '|' + R[1] + R[0])
# SUBSTITUTION
# [('', 'hte'), ('h', 'te'), ('ht', 'e'), ('hte', '')]
# e|i 
# R[0] + '|' + c
# 
#     replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    for L, R in splits:
        if R:
            for c in letters:
                replaces.append(L + c + R[1:])
                replaceEdits.append(R[0] + '|' + c)
    
# INSERTION
# [('', 'hte'), ('h', 'te'), ('ht', 'e'), ('hte', '')]
# 
# >|>x if not L '>|>' + c
# h|ha str(L[-1]) + '|' + str(L[-1]) + c
#     inserts    = [L + c + R               for L, R in splits for c in letters]
    for L, R in splits:
        for c in letters:
            inserts.append(L + c + R)
            if not L:
                insertEdits.append('>|>' + c)
            else:
                insertEdits.append(str(L[-1]) + '|' + str(L[-1]) + c)
    
    print(deleteEdits)
    print(insertEdits)
    print(replaceEdits)
    print(transposeEdits)
    
    wordsAndEdits = {
        'del': deletes,
        'tra': transposes,
        'sub': replaces,
        'ins': inserts,
        'delEdits': deleteEdits,
        'traEdits': transposeEdits,
        'subEdits': replaceEdits,
        'insEdits': insertEdits
    }
#     return set(deletes + transposes + replaces + inserts)
    return wordsAndEdits

In [4]:
def spellCorrect(word):
    # check if the word is in the corpus
    if word in words:
        return "No error"

    # word should be corrected
    # get possible words within edit distance 1
    candidates = edits1(word)

    candidatesProb = {
        'del': [],
        'tra': [],
        'sub': [],
        'ins': []
    }
#     print(wordsAndEdits)
    edit_types = ['del', 'tra', 'sub', 'ins']
    
# find the probability of the candidate in the corpus
    for etype in edit_types:
        for candidate in candidates[etype]:
            if candidate in words:
                candidatesProb[etype].append(all_words[candidate] / total_types)
            else:
                candidatesProb[etype].append(0)

    temp = []
# remove the candidates with 0 probability
    for etype in edit_types:
        i = 0    
        while i != len(candidates[etype]):
            if candidatesProb[etype][i] == 0:
                del candidates[etype][i]
                del candidatesProb[etype][i]
                del candidates[etype + 'Edits'][i]
            else:
                temp.append([word, candidates[etype][i], etype, candidates[etype + 'Edits'][i], candidatesProb[etype][i], -1, -1])
                i += 1
                
#     print(candidates)
#     print(candidatesProb)
#     print(temp)
    return temp

In [5]:
word = input("Input: ")

# call functions, return answer
output = spellCorrect(word)
print("Output: ")
headers = ['word', 'candidate', 'edit_type', 'edit', 'P(c)', 'P(w|c)', 'P(c) x P(w|c)']
display(pandas.DataFrame(output, columns = headers))

Input: suceeded
['>s|>', 'su|s', 'uc|u', 'ce|c', 'ee|e', 'ed|e', 'de|d', 'ed|e']
['>|>a', '>|>b', '>|>c', '>|>d', '>|>e', '>|>f', '>|>g', '>|>h', '>|>i', '>|>j', '>|>k', '>|>l', '>|>m', '>|>n', '>|>o', '>|>p', '>|>q', '>|>r', '>|>s', '>|>t', '>|>u', '>|>v', '>|>w', '>|>x', '>|>y', '>|>z', '>|>-', ">|>'", 's|sa', 's|sb', 's|sc', 's|sd', 's|se', 's|sf', 's|sg', 's|sh', 's|si', 's|sj', 's|sk', 's|sl', 's|sm', 's|sn', 's|so', 's|sp', 's|sq', 's|sr', 's|ss', 's|st', 's|su', 's|sv', 's|sw', 's|sx', 's|sy', 's|sz', 's|s-', "s|s'", 'u|ua', 'u|ub', 'u|uc', 'u|ud', 'u|ue', 'u|uf', 'u|ug', 'u|uh', 'u|ui', 'u|uj', 'u|uk', 'u|ul', 'u|um', 'u|un', 'u|uo', 'u|up', 'u|uq', 'u|ur', 'u|us', 'u|ut', 'u|uu', 'u|uv', 'u|uw', 'u|ux', 'u|uy', 'u|uz', 'u|u-', "u|u'", 'c|ca', 'c|cb', 'c|cc', 'c|cd', 'c|ce', 'c|cf', 'c|cg', 'c|ch', 'c|ci', 'c|cj', 'c|ck', 'c|cl', 'c|cm', 'c|cn', 'c|co', 'c|cp', 'c|cq', 'c|cr', 'c|cs', 'c|ct', 'c|cu', 'c|cv', 'c|cw', 'c|cx', 'c|cy', 'c|cz', 'c|c-', "c|c'", 'e|ea', 'e|eb', 'e|ec'

,word,candidate,edit_type,edit,P(c),P(w|c),P(c) x P(w|c)
0,suceeded,succeeded,ins,u|uc,0.001394,-1,-1
1,suceeded,succeeded,ins,c|cc,0.001394,-1,-1


# **Your Relfection / Takeaway / Analysis**

*Kindly place the rest of your write up. More information is found in the Canvas write up.*

- Briefly describe the approach you took to implementing the spell correction assessment (both for detecting an error and correcting an error)
- Did you make any modifications based from the method described in class?
- What is an ideal corpus / edit distance matrix in relation to spell correction?
- Do you think the corpus / edit distance matrix you used is good in relation to the task? Are there issues that you observed in using the data that might limit the performance of the model?
- Are there ways we could modify the corpus / edit distance matrix to make the model more accommodating?
- What can / can't your model do well? You may include examples.
- Please note that you're not expected to create a perfect spell correction model, so there's no need to force the model to accommodate all known bugs of the model.
- Do you think / can you imagine cases that the Noisy Channel cannot handle no matter what you do?

- Briefly describe the approach you took to implementing the spell correction assessment (both for detecting an error and correcting an error)

The approach that we took was to first check if the word was in our corpus so that we would know if the word has a spelling error or none. If there were to be a spelling error, then we would check for all of the possible edits within one edit distance and store the edit that happened. After this, we checked the candidates with our corpus to see if the words were part of the 'correct' words. It should be noted that since the corpus only consists of a number of words, it will not have all the words we currently use. After having the set of candidates, their edits, and probability of occuring in the corpus, we check the edits that were done to the single-edit spelling correction edits based off of Peter Norvig's collection of spelling errors. If there were edits that occured that did not appear in the collection, it would be removed from being a candidate. The remaining candidates would be used to show the possible correct spelling for the input while 'No error' would be the output if there were to be no error or candidates were not in the collection of spelling errors.

- Did you make any modifications based from the method described in class?

I think we did a similar method described in class where we used the Damerau-Levenshtein Edit Distance to find the edits with distance 1, its type of correction and candidate word. With the language model for the candidate word, we were able to use it with the channel model form the edits and Peter Norvig's collection of spelling errors to produce the noisy channel model.

- What is an ideal corpus / edit distance matrix in relation to spell correction?

An ideal corpus would be an English dictionary for English words because it would have the correct spelling of all the words and have a large number of words be in the corpus.

- Do you think / can you imagine cases that the Noisy Channel cannot handle no matter what you do?

Maybe words that are not in different corpora yet that are real words. Maybe before when selfie was not yet declared a word, it would have shown that it was a spelling error.